# FETCH3.14

This notebook documents the process of running the FETCH3.14 model using BOA. Because FETCH3.14 model is more difficult to install than SWAT+ currently (SWAT+ is a single not very large executable), it is not included in this repository, so only instructions on how to run it are provided. All the necessary files are included in the repository, so you can run the model on your own machine or on a SLURM cluster.

FETCH3.14 has first class support for Bayesian optimization through BOA. This means that you can run FETCH3.14 with BOA to optimize your FETCH3.14 model with BO without needing to write any additional code. This is done directly in the FETCH3.14 configuration file giving users a convenient and seamless experience.

In [1]:
# This notebook uses hidden cells to import and display things so
# that it can be ran regullary to make sure the documentation
# is up to date and not broken (as opposed to a markdown file
# that would have an exmaple written once and get out of date
# as the code base changes)

# These hidden cells are only responsible for rerunning
# the documentation to ensure it is correct.
# the actual relavent part of the documentation
# are the non hidden parts

In [3]:
import pathlib

from IPython.display import Code

boa_paper_dir = pathlib.Path().resolve().parent
r_dir = boa_paper_dir / "fetch3"
config_path = r_dir / "boa_moo_2_obj_norm_2015_scale.yaml"

## Configuration File

The FETCH3.14 and BOA boa_moo_2_obj_norm_2015_scale.yaml in one

In [4]:
Code(config_path)

{% set fetch_run_time = '1:30:00' %}
{% set model_dir = '.' %}
{% set data_path = './data/' %}
{% set model_trees = ({
  'USA_UMB_CON_Aru_Js_28': {
    'Hspec': 17.5,
    'dbh': 10.2,
    'mean_crown_area_sp': 34.16,
    'sapwood_depth': 2.669080975,
    'scale_nhl': 0.7854047094285671}
}) %}
{% set tree_names = model_trees.keys()|list %}
{% set hpft = 'maple' %}

objective:
  metrics:
    {% for tree in tree_names %}
    - metric: "RootMeanSquaredError"
      name: rmse_{{ tree }}_sapflux
      properties:
        obs_file: {{ data_path }}USA_UMB_CON_sapf_data_tz.csv
        obs_var: {{ tree }}
        output_fname: ds_sapflux.nc
        output_var: {{ tree }}
        fetch_data_func: get_model_sapflux
        normalize: True
        scaling_factor: 1.1
        hour_range: [9, 21]
        minimize: True
    - metric: "RootMeanSquaredError"
      name: rmse_{{ tree }}_wwc
      properties:
        obs_file: {{ data_path }}wwc_trees_2016.csv
        obs_var: m9b
        output_fname: ds_sapflux.nc
        output_var: theta
        species: {{ tree }}
        normalize: True
        fetch_data_func: get_model_obs
        scaling_factor: 1.1
        hour_range: [9, 21]
        minimize: True
    {% endfor %}
scheduler:
  n_trials: 200
  tolerated_trial_failure_rate: 0.5
  ttl_seconds_for_trials: 6000
  run_trials_in_batches: true
  max_pending_trials: 10
generation_strategy:
  num_initialization_trials: 20
  max_parallelism_override: 10

script_options:
  exp_name: fetch3_moo_2_obj_norm_2015_scale
  output_dir: ./output

  run_cmd: python main.py --config_path {config_path} --data_path {data_path} --output_path {trial_dir}
#  run_cmd: sbatch --output={trial_dir}/slurm_log_%j.log --time={{ fetch_run_time }} {{ model_dir }}/scripts/slurm_main.sh {config_path} {data_path} {trial_dir}

  wrapper_path: {{ model_dir }}/fetch3/optimize/fetch_wrapper.py
  wrapper_name: Fetch3Wrapper
  rel_to_launch: true

model_options:
  data_path: {{ data_path }}

  make_experiment_dir: False
  # File for input met data
  input_fname: "umbs_flux_adjustedP5.csv"
  # input_fname: FLX_US-UMB_FLUXNET2015_SUBSET_HH_2007-2017_beta-4.csv
  met_column_labels:
    CO2_F_MDS: CO2_F

  # Start and end for the model
  start_time: "2016-05-29 00:00:00"
  end_time: "2016-08-05 00:00:00"

  dt:  1800  #seconds - input data resolution
  tmin:  0  #tmin [s]

  # Site information
  latitude:  45.5598  # From AmeriFlux UMBS page
  longitude:  -84.7138
  time_offset:  -5 #Offset from UTC time, e.g EST = UTC -5 hrs

  # Run options

  # Printing slows down model run
  # Options to turn printing off or specify print frequency
  print_run_progress:  True  # Turn on/off printing for progress of time steps calculated
  print_freq:  2000  # Interval of timesteps to print if print_run_progress = True (e.g. 1 will print every time step)

  # Transpiration options: NHL or PM transpiration scheme
  transpiration_scheme:  1 # 0: PM transpiration; 1: NHL transpiration

  # Numerical solution time and space constants
  #The finite difference discretization constants
  dt0: 20  #model temporal resolution [s]
  dz: 0.2  #model spatial resolution [m]
  stop_tol: .0001  #0.0001  #stop tolerance of equation converging

  # Soil boundary conditions

  # Upper Boundary condition
  # 1 = no flux (Neuman)
  # 0 = infiltration

  # Bottom Boundary condition
  # 2 = free drainage
  # 1 = no flux (Neuman)
  # 0 = constant potential (Dirichlet)

  UpperBC: 0
  BottomBC: 0

  # Tree information
  LAD_norm:  'LAD_data.csv' # File with LAD data
  LAD_column_labels:
    {% for tree in tree_names %}
    {{ tree }}: {{ hpft }}
    {% endfor %}

  # Leaf area density formulation
  lad_scheme:  1  #0: default scheme, based on Lalic et al 2014; 1: scheme from NHL module

groups:
  site_parameters:
    Soil_depth: 6.0  # [m] depth of soil column
    sand_d: 6.0 #4.2----top soil #m
    clay_d: 1.0 #0------4.2 #m

    # Soil initial conditions
    initial_swc_clay: 0.28 #from Verma et al

    initial_swc_sand

## Run Model Wrapper Script

FETCH3.14 does not have a run model script because it utilizes the BOA python API to run the model. This is done by overriding the BOA wrapper methods to control the model execution. FETCH3.14 then specifies to BOA how to find this subclassed wrapper under the `script_options` section in the above config. All of this is directly part of the FETCH3.14 model, so the user does not need to worry about it giving FETCH3.14 first class support through BOA to Bayesian optimization, and allowing users to access that optimization through their normal FETCH3.14 configuration. 

## Running our script

To run our script we would just need to pass the config file to BOA's CLI

```python
python -m boa --config-file path/to/config.yaml
```

or

```python
python -m boa -c path/to/config.yaml
```